# Рекомендация тарифов мобильной связи

Оператор мобильной связи выяснил, что многие клиенты пользуются архивными тарифами. Необходимо  построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Предоставлены данные о поведении клиентов, которые перешли на тарифы «Смарт» и «Ультра». Нужно построить модель для задачи классификации, которая выберет подходящий из этих тариф. Данные уже были предобработаны в аналитическом проекте `cellular operator tariffs analysis`.

Необходимо постройть модель с максимально большим значением *accuracy*, минимальная доля правильных ответов - 0.75. 

В наборе данных информация о поведении одного пользователя за месяц. Известно:  
`сalls` — количество звонков,  
`minutes` — суммарная длительность звонков в минутах,  
`messages` — количество sms-сообщений,  
`mb_used` — израсходованный интернет-трафик в Мб,  
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

#### План действий:  
1) Откроем файл с данными `datasets/users_behavior.csv` и изучим его.   
2) Разделим исходные данные на обучающую, валидационную и тестовую выборки.  
3) Исследуем качество разных моделей, меняя гиперпараметры.  
4) Проверим качество модели на тестовой выборке.  
5) Проверим модели на вменяемость.    

## 1. Откроем файлы с данными и изучим информацию


Импортируем все необходимые для анализа и обучения библиотеки. Откроем файл.

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

import warnings # отключение предупреждений
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('datasets/users_behavior.csv')
data

,Unnamed: 0,calls,minutes,messages,mb_used,is_ultra
0,0,40.0,311.90,83.0,19915.42,0
1,1,85.0,516.75,56.0,22696.96,0
2,2,77.0,467.66,86.0,21060.45,0
3,3,106.0,745.53,81.0,8437.39,1
4,4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...,...
3209,3209,122.0,910.98,20.0,35124.90,1
3210,3210,25.0,190.36,0.0,3275.61,0
3211,3211,97.0,634.44,70.0,13974.06,0
3212,3212,64.0,462.32,90.0,31239.78,0


В нашей выборке представлено 3214 объектов и 5 признаков. Целевой признак, который нужно предсказать, — это тариф (`is_ultra`).

В данном случае целевой признак категориальный, будем решаеть задачу бинарной классификации выбора тарифа — «Ультра» или «Смарт».

## 2. Разбиваем данные на выборки

- Разобьем наш датасет на на обучающую, валидационную и тестовую выборки при помощи NumPy.split в пропорции 60%-20%-20%.
- Разделим их на признаки.

In [149]:
train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))]) # разбивка 60%-20%-20%

In [150]:
features_train = train.drop(['is_ultra'], axis=1) # тренировочная выборка
target_train = train['is_ultra']

features_validate = validate.drop(['is_ultra'], axis=1) # валидационная выборка
target_validate = validate['is_ultra']

features_test = test.drop(['is_ultra'], axis=1) # тестовая выброка
target_test = test['is_ultra']

## 3. Исследуйте модели

Используем для обучения модели следующие алгоритмы:

- DecisionTreeClassifier
- RandomForestClassifier
- LogisticRegression.

Обучим модель и проверим алгоритмы на валидационной выборке. 

Чтобы улучшить правильность предсказаний нашей модели необходимо подобрать соответствующий гиперпараметр `maх_depth` и `n_estimators`, при этом соблюсти баланс между переобучением и недообучением. Для автоматизации процесса, создадим цикл подставляющий разные значения и сравним качество моделей в разных вариантах.

In [151]:
for mxd in range(1, 16): # DecisionTreeClassifier
    model = DecisionTreeClassifier(random_state=12345, max_depth = mxd)
    model.fit(features_train, target_train)
    test_predictions_dtc = model.predict(features_validate)
    test_accuracy_dtc = accuracy_score(target_validate, test_predictions_dtc)
    print('max_depth =', mxd, ':', test_accuracy_dtc)

max_depth = 1 : 0.7433903576982893
max_depth = 2 : 0.776049766718507
max_depth = 3 : 0.7916018662519441
max_depth = 4 : 0.776049766718507
max_depth = 5 : 0.7729393468118196
max_depth = 6 : 0.7869362363919129
max_depth = 7 : 0.7931570762052877
max_depth = 8 : 0.7838258164852255
max_depth = 9 : 0.7978227060653188
max_depth = 10 : 0.7947122861586314
max_depth = 11 : 0.7916018662519441
max_depth = 12 : 0.7807153965785381
max_depth = 13 : 0.7651632970451011
max_depth = 14 : 0.7511664074650077
max_depth = 15 : 0.7480559875583204


In [152]:
for nest in range(1, 16): # RandomForestClassifier
    model = RandomForestClassifier(random_state=12345, n_estimators=nest)
    model.fit(features_train, target_train)
    test_predictions_rfc = model.predict(features_validate)
    test_accuracy_rfc = accuracy_score(target_validate, test_predictions_rfc)
    print('n_estimator =', nest, ':', test_accuracy_rfc)

n_estimator = 1 : 0.7325038880248833
n_estimator = 2 : 0.7822706065318819
n_estimator = 3 : 0.7744945567651633
n_estimator = 4 : 0.7853810264385692
n_estimator = 5 : 0.7651632970451011
n_estimator = 6 : 0.7822706065318819
n_estimator = 7 : 0.7884914463452566
n_estimator = 8 : 0.7962674961119751
n_estimator = 9 : 0.7791601866251944
n_estimator = 10 : 0.7931570762052877
n_estimator = 11 : 0.7744945567651633
n_estimator = 12 : 0.7807153965785381
n_estimator = 13 : 0.7791601866251944
n_estimator = 14 : 0.7869362363919129
n_estimator = 15 : 0.7807153965785381


In [153]:
model_lr = LogisticRegression(random_state=12345) # LogisticRegression
model_lr.fit(features_train, target_train)
test_predictions_lr = model_lr.predict(features_validate)
test_accuracy_lr = accuracy_score(target_validate, test_predictions_lr)
test_accuracy_lr

0.6858475894245724

Максимальная доля правильных ответов достигается при глубине `max_depth` равной 3-4 и при количестве оценщиков `n_estimators` - 9-10. При этом лучше работают модели DecisionTreeClassifier и RandomForestClassifier. 

## 4. Проверка моделей на тестовой выборке

Использум параметры `max_depth` и `n_estimators`, показавшие максимальные доли правильных ответов.

In [154]:
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth = 4)
model_dtc.fit(features_train, target_train)
model_dtc.score(features_test, target_test)

0.8118195956454122

In [155]:
model_rfc = RandomForestClassifier(random_state=12345, n_estimators=9)
model_rfc.fit(features_train, target_train)
model_rfc.score(features_test, target_test)

0.8133748055987559

In [156]:
model_lr.score(features_test, target_test)

0.7262830482115086

Итак, тестовая выборка подтвердила правильность выбора алгоритма обучения - DecisionTreeClassifier и RandomForestClassifier. Мы достигли значения доли правильных ответов (accuracy) - 0.78-0.8.   

## 5. (бонус) Проверка модели на адекватность

Используем классификатор DummyClassifier в качестве сравнения с выбранным алгоритмом модели. Стратегия (`strategy`) будет генерировать прогнозы равномерно случайным образом - «uniform».

In [157]:
dummy_clf = DummyClassifier(strategy='uniform')
dummy_clf.fit(features_train, target_train)
# DummyClassifier(strategy='most_frequent')
dummy_clf.predict(features_test)
print(dummy_clf.score(features_test, target_test), 'против', model_dtc.score(features_test, target_test))

0.5334370139968896 против 0.8118195956454122


Наша модель работает явно лучше!